In [1]:
# ============================================
# ScoresAndOdds NCAAB Scraper — Standalone Jupyter Cell
# - Fetches https://www.scoresandodds.com/ncaab
# - Combines the 2-row game blocks (23 games -> 46 team rows typical)
# - Parses OPEN vs CURRENT spread/total from "Line Movements" (first token = open, last token = current)
# - Writes:
#     1) daily append log: market_whisker_guard_YYYY-MM-DD.csv
#     2) latest snapshot:  market_whisker_guard_latest.csv
# - Safe if Excel locks a file (falls back to a timestamped filename)
# ============================================

import re
from datetime import datetime, date
from pathlib import Path
from io import StringIO

import requests
import pandas as pd

# ---------- SETTINGS ----------
URL = "https://www.scoresandodds.com/ncaab"

# folder where CSVs will be written (current folder by default)
OUT_DIR = Path(".")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CAT_BASENAME = "market_whisker_guard"

DAILY_FILE  = OUT_DIR / f"{CAT_BASENAME}_{date.today().isoformat()}.csv"
LATEST_FILE = OUT_DIR / f"{CAT_BASENAME}_latest.csv"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

BASE_COLS = ["Unnamed: 0", "Open", "Line Movements", "Spread", "Total", "Moneyline", "Notes"]


# ---------- FETCH ----------
def fetch_html(url: str, timeout: int = 30) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text


# ---------- PARSERS ----------
def normalize_first_col(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    # Some blocks use FINAL / Unnamed: 1; normalize to Unnamed: 0
    if "FINAL" in df.columns:
        df = df.rename(columns={"FINAL": "Unnamed: 0"})
    if "Unnamed: 1" in df.columns and "Unnamed: 0" not in df.columns:
        df = df.rename(columns={"Unnamed: 1": "Unnamed: 0"})
    return df

def parse_rotation_team(text: str):
    """
    '863 South Dakota State 7-6' -> (863, 'South Dakota State', '7-6')
    """
    text = str(text).replace("\n", " ").strip()
    m = re.match(r"^(\d+)\s+(.*)\s+(\d{1,2}-\d{1,2})$", text)
    if not m:
        return None, text, None
    return int(m.group(1)), m.group(2).strip(), m.group(3).strip()

def extract_spread_tokens(line_movements: str):
    """
    Strictly match spread tokens that look like:
      -1.5 -108   or   +3.0 -112
    (We try to avoid grabbing totals like 149.5 -110 by requiring a leading +/-)
    """
    s = "" if line_movements is None else str(line_movements).replace("\n", " ")
    return re.findall(r"([+-]\d+(?:\.\d+)?)\s*([+-]?\d+)", s)

def extract_total_tokens(line_movements: str):
    """
    Match totals tokens that look like:
      o149.5 -110   or   u152.5 -105
    """
    s = "" if line_movements is None else str(line_movements).replace("\n", " ")
    return re.findall(r"([ou])\s*(\d+(?:\.\d+)?)\s*([+-]?\d+)", s, flags=re.IGNORECASE)

def first_last_spread(line_movements: str):
    toks = extract_spread_tokens(line_movements)
    if not toks:
        return (None, None), (None, None)
    first = (float(toks[0][0]), int(toks[0][1]))
    last  = (float(toks[-1][0]), int(toks[-1][1]))
    return first, last

def first_last_total(line_movements: str):
    toks = extract_total_tokens(line_movements)
    if not toks:
        return (None, None, None), (None, None, None)
    f = toks[0]; l = toks[-1]
    first = (f[0].lower(), float(f[1]), int(f[2]))
    last  = (l[0].lower(), float(l[1]), int(l[2]))
    return first, last

def parse_moneyline_cell(s: str):
    s = "" if s is None else str(s).replace("\n", " ").strip()
    m = re.search(r"([+-]?\d+)", s)
    return int(m.group(1)) if m else None


# ---------- BUILD MARKET SNAPSHOT ----------
def build_market_snapshot_from_tables(tables: list[pd.DataFrame]) -> pd.DataFrame:
    # Keep only the 2-row game blocks that match the odds schema
    parts = []
    for t in tables:
        t = normalize_first_col(t)
        if all(c in t.columns for c in BASE_COLS) and len(t) == 2:
            parts.append(t[BASE_COLS])

    if not parts:
        raise RuntimeError("No 2-row game tables found. Page layout may have changed.")

    slate_raw = pd.concat(parts, ignore_index=True)
    games = len(slate_raw) // 2

    rows = []
    for i in range(0, len(slate_raw), 2):
        a = slate_raw.iloc[i]
        b = slate_raw.iloc[i + 1]

        rot_a, team_a, rec_a = parse_rotation_team(a["Unnamed: 0"])
        rot_b, team_b, rec_b = parse_rotation_team(b["Unnamed: 0"])

        (open_sp_a, open_sp_price_a), (cur_sp_a, cur_sp_price_a) = first_last_spread(a["Line Movements"])
        (open_sp_b, open_sp_price_b), (cur_sp_b, cur_sp_price_b) = first_last_spread(b["Line Movements"])

        (open_tot_side_a, open_tot_a, open_tot_price_a), (cur_tot_side_a, cur_tot_a, cur_tot_price_a) = first_last_total(a["Line Movements"])
        (open_tot_side_b, open_tot_b, open_tot_price_b), (cur_tot_side_b, cur_tot_b, cur_tot_price_b) = first_last_total(b["Line Movements"])

        game_id = f"{rot_a}-{rot_b}"

        rows.append({
            "game_id": game_id,
            "rotation": rot_a,
            "opponent_rotation": rot_b,
            "team": team_a,
            "record": rec_a,

            "open_spread": open_sp_a,
            "open_spread_price": open_sp_price_a,
            "current_spread": cur_sp_a,
            "current_spread_price": cur_sp_price_a,

            "open_total": open_tot_a,
            "open_total_side": open_tot_side_a,
            "open_total_price": open_tot_price_a,
            "current_total": cur_tot_a,
            "current_total_side": cur_tot_side_a,
            "current_total_price": cur_tot_price_a,

            "current_moneyline": parse_moneyline_cell(a["Moneyline"]),
        })

        rows.append({
            "game_id": game_id,
            "rotation": rot_b,
            "opponent_rotation": rot_a,
            "team": team_b,
            "record": rec_b,

            "open_spread": open_sp_b,
            "open_spread_price": open_sp_price_b,
            "current_spread": cur_sp_b,
            "current_spread_price": cur_sp_price_b,

            "open_total": open_tot_b,
            "open_total_side": open_tot_side_b,
            "open_total_price": open_tot_price_b,
            "current_total": cur_tot_b,
            "current_total_side": cur_tot_side_b,
            "current_total_price": cur_tot_price_b,

            "current_moneyline": parse_moneyline_cell(b["Moneyline"]),
        })

    df = pd.DataFrame(rows)
    df["captured_at"] = datetime.now().isoformat(timespec="seconds")

    print(f"✅ Found {games} games ({len(df)} team rows)")
    return df


# ---------- WRITE FILES (safe with Excel locks) ----------
def safe_append(df: pd.DataFrame, path: Path) -> Path:
    """
    Append to CSV; if locked (Excel), write to a timestamped fallback file.
    """
    try:
        df.to_csv(path, mode="a", header=not path.exists(), index=False)
        return path
    except PermissionError:
        ts = datetime.now().isoformat(timespec="seconds").replace(":", "-")
        fallback = path.with_name(path.stem + f"_{ts}" + path.suffix)
        df.to_csv(fallback, index=False)
        return fallback


# ---------- RUN ----------
html = fetch_html(URL)
tables = pd.read_html(StringIO(html))

market = build_market_snapshot_from_tables(tables)

# daily append log
daily_written = safe_append(market, DAILY_FILE)

# latest overwrite
market.to_csv(LATEST_FILE, index=False)

print("\nSaved:")
print("  DAILY  ->", daily_written.resolve())
print("  LATEST ->", LATEST_FILE.resolve())

display(market.head(12))


✅ Found 13 games (26 team rows)

Saved:
  DAILY  -> C:\Users\gribb\market_whisker_guard_2025-12-19.csv
  LATEST -> C:\Users\gribb\market_whisker_guard_latest.csv


,game_id,rotation,opponent_rotation,team,record,open_spread,open_spread_price,current_spread,current_spread_price,open_total,open_total_side,open_total_price,current_total,current_total_side,current_total_price,current_moneyline,captured_at
0,509-510,509,510,Norfolk State,4-8,-5.5,-112,-5.5,-108,NaN,None,NaN,NaN,None,NaN,-245.0,2025-12-19T16:42:02
1,509-510,510,509,Jackson State,1-10,-10.0,8,-10.0,8,139.5,u,-108.0,139.5,o,-108.0,200.0,2025-12-19T16:42:02
2,873-874,873,874,Villanova,8-2,-11.0,2,-11.0,2,152.5,o,-112.0,152.5,u,-112.0,154.0,2025-12-19T16:42:02
3,873-874,874,873,Wisconsin,7-3,-3.5,-115,-4.5,-105,NaN,None,NaN,NaN,None,NaN,-185.0,2025-12-19T16:42:02
4,511-512,511,512,Alcorn State,1-9,-11.0,0,-11.0,0,160.5,o,-110.0,160.5,o,-110.0,NaN,2025-12-19T16:42:02
5,511-512,512,511,Baylor,7-2,-35.5,-105,-35.5,-108,NaN,None,NaN,NaN,None,NaN,NaN,2025-12-19T16:42:02
6,513-514,513,514,Florida A&M,3-5,-11.0,2,-11.0,5,145.5,o,-112.0,145.5,o,-115.0,800.0,2025-12-19T16:42:02
7,513-514,514,513,Tarleton State,8-4,-14.5,-105,-14.5,-105,NaN,None,NaN,NaN,None,NaN,-1350.0,2025-12-19T16:42:02
8,515-516,515,516,Southeastern Louisiana,4-7,-11.0,5,-11.0,2,144.5,o,-115.0,144.5,u,-112.0,2500.0,2025-12-19T16:42:02
9,515-516,516,515,LSU,9-1,-24.5,-105,-24.5,-105,NaN,None,NaN,NaN,None,NaN,-9000.0,2025-12-19T16:42:02
